In [3]:
import pandas as pd
from tsai.all import *
computer_setup()

os             : Linux-5.3.0-40-generic-x86_64-with-glibc2.10
python         : 3.8.2
tsai           : 0.3.1
fastai         : 2.6.0
fastcore       : 1.4.2
torch          : 1.11.0+cu102
device         : 1 gpu (['TITAN RTX'])
cpu cores      : 16
RAM            : 62.75 GB
GPU memory     : [23.65] GB


# Preprocessing

## Charts Data

In [192]:
df_usa = pd.read_csv('USA.csv', index_col=0)

In [193]:
df_usa.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 454832 entries, 0 to 454831
Data columns (total 26 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   title             454832 non-null  object 
 1   rank              454832 non-null  int64  
 2   date              454832 non-null  object 
 3   artist            454832 non-null  object 
 4   url               454832 non-null  object 
 5   region            454832 non-null  object 
 6   chart             454832 non-null  object 
 7   trend             454832 non-null  object 
 8   streams           364157 non-null  float64
 9   id                454832 non-null  object 
 10  danceability      454832 non-null  float64
 11  energy            454832 non-null  float64
 12  key               454832 non-null  float64
 13  loudness          454832 non-null  float64
 14  mode              454832 non-null  float64
 15  speechiness       454832 non-null  float64
 16  acousticness      45

In [194]:
df_usa.nunique()

title                11837
rank                   200
date                  1826
artist                5209
url                  14644
region                   1
chart                    2
trend                    4
streams             243632
id                   14644
danceability           793
energy                 945
key                     12
loudness              7202
mode                     2
speechiness           1297
acousticness          2736
instrumentalness      3202
liveness              1322
valence               1194
tempo                10986
uri                  14644
track_href           14644
analysis_url         14644
duration_ms          11378
time_signature           4
dtype: int64

In [195]:
df_usa_ids = pd.DataFrame(df_usa['id'].unique(), columns=['id'])

We drop most categorical features.

In [196]:
df_usa.drop(columns=['url', 'uri', 'trend', 'track_href', 'analysis_url', 'region', 'streams', 'title', 'artist','id'], inplace=True)

In [197]:
df_usa

,rank,date,chart,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,1,2017-01-01,top200,0.926,0.666,11.0,-5.314,1.0,0.2440,0.0611,0.000000,0.1230,0.168,127.079,343150.0,4.0
1,2,2017-01-01,top200,0.928,0.481,9.0,-9.350,0.0,0.2870,0.1050,0.000000,0.1760,0.613,134.007,210937.0,4.0
2,3,2017-01-01,top200,0.681,0.594,7.0,-7.028,1.0,0.2820,0.1650,0.000003,0.1340,0.535,186.054,230453.0,4.0
3,4,2017-01-01,top200,0.748,0.524,8.0,-5.599,1.0,0.0338,0.4140,0.000000,0.1110,0.661,95.010,244960.0,4.0
4,5,2017-01-01,top200,0.794,0.632,0.0,-6.163,1.0,0.0649,0.1420,0.000000,0.1280,0.355,145.926,291893.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
454827,46,2021-07-31,viral50,0.808,0.897,11.0,-3.712,0.0,0.0348,0.0469,0.000031,0.3640,0.591,126.026,231041.0,4.0
454828,47,2021-07-31,viral50,0.499,0.695,6.0,-9.452,1.0,0.1130,0.3330,0.000003,0.6590,0.235,131.006,183920.0,4.0
454829,48,2021-07-31,viral50,0.586,0.559,4.0,-6.725,1.0,0.0500,0.6390,0.000000,0.0872,0.437,177.945,167480.0,4.0
454830,49,2021-07-31,viral50,0.699,0.654,11.0,-6.264,1.0,0.0383,0.0540,0.000007,0.0950,0.319,100.037,192313.0,4.0


We spilt based on the chart.

In [198]:
df_usa_200 = pd.DataFrame(df_usa[df_usa['chart'] == 'top200'])
df_usa_200.drop(columns=['chart'], inplace=True)
df_usa_200

,rank,date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,1,2017-01-01,0.926,0.666,11.0,-5.314,1.0,0.2440,0.0611,0.000000,0.1230,0.168,127.079,343150.0,4.0
1,2,2017-01-01,0.928,0.481,9.0,-9.350,0.0,0.2870,0.1050,0.000000,0.1760,0.613,134.007,210937.0,4.0
2,3,2017-01-01,0.681,0.594,7.0,-7.028,1.0,0.2820,0.1650,0.000003,0.1340,0.535,186.054,230453.0,4.0
3,4,2017-01-01,0.748,0.524,8.0,-5.599,1.0,0.0338,0.4140,0.000000,0.1110,0.661,95.010,244960.0,4.0
4,5,2017-01-01,0.794,0.632,0.0,-6.163,1.0,0.0649,0.1420,0.000000,0.1280,0.355,145.926,291893.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
441639,196,2018-01-31,0.835,0.413,1.0,-9.810,1.0,0.3960,0.3730,0.001330,0.1010,0.111,149.004,258880.0,4.0
441640,197,2018-01-31,0.741,0.761,4.0,-3.871,0.0,0.0512,0.2350,0.000000,0.1190,0.217,81.543,149576.0,4.0
441641,198,2018-01-31,0.766,0.709,9.0,-6.471,0.0,0.1230,0.2040,0.000014,0.1260,0.506,128.070,211853.0,4.0
441642,199,2018-01-31,0.734,0.418,0.0,-6.678,1.0,0.0425,0.0129,0.000000,0.0579,0.868,85.909,188174.0,4.0


In [199]:
df_usa_50 = pd.DataFrame(df_usa[df_usa['chart'] == 'viral50'])
df_usa_50.drop(columns=['chart'], inplace=True)
df_usa_50

,rank,date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
800,32,2017-08-09,0.595,0.610,1.0,-7.007,0.0,0.0330,0.0910,0.000000,0.1600,0.5040,99.529,218319.0,4.0
801,1,2017-08-01,0.810,0.342,1.0,-8.801,1.0,0.0892,0.7020,0.027400,0.1100,0.0881,77.515,199742.0,4.0
802,2,2017-08-01,0.653,0.910,10.0,-3.360,1.0,0.0566,0.0079,0.000000,0.3180,0.0921,126.047,234286.0,4.0
803,3,2017-08-01,0.922,0.583,5.0,-4.982,1.0,0.0753,0.0098,0.000000,0.0962,0.4940,136.043,261615.0,4.0
804,4,2017-08-01,0.867,0.545,2.0,-5.192,1.0,0.0625,0.0646,0.000289,0.0505,0.5250,139.943,162586.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
454827,46,2021-07-31,0.808,0.897,11.0,-3.712,0.0,0.0348,0.0469,0.000031,0.3640,0.5910,126.026,231041.0,4.0
454828,47,2021-07-31,0.499,0.695,6.0,-9.452,1.0,0.1130,0.3330,0.000003,0.6590,0.2350,131.006,183920.0,4.0
454829,48,2021-07-31,0.586,0.559,4.0,-6.725,1.0,0.0500,0.6390,0.000000,0.0872,0.4370,177.945,167480.0,4.0
454830,49,2021-07-31,0.699,0.654,11.0,-6.264,1.0,0.0383,0.0540,0.000007,0.0950,0.3190,100.037,192313.0,4.0


We set the date as the index

In [200]:
df_usa_50['date'] = pd.to_datetime(df_usa_50['date'])

In [201]:
df_usa_50_dex = df_usa_50.set_index('date')

In [202]:
df_usa_50_dex

,rank,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
date,,,,,,,,,,,,,,
2017-08-09,32,0.595,0.610,1.0,-7.007,0.0,0.0330,0.0910,0.000000,0.1600,0.5040,99.529,218319.0,4.0
2017-08-01,1,0.810,0.342,1.0,-8.801,1.0,0.0892,0.7020,0.027400,0.1100,0.0881,77.515,199742.0,4.0
2017-08-01,2,0.653,0.910,10.0,-3.360,1.0,0.0566,0.0079,0.000000,0.3180,0.0921,126.047,234286.0,4.0
2017-08-01,3,0.922,0.583,5.0,-4.982,1.0,0.0753,0.0098,0.000000,0.0962,0.4940,136.043,261615.0,4.0
2017-08-01,4,0.867,0.545,2.0,-5.192,1.0,0.0625,0.0646,0.000289,0.0505,0.5250,139.943,162586.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-07-31,46,0.808,0.897,11.0,-3.712,0.0,0.0348,0.0469,0.000031,0.3640,0.5910,126.026,231041.0,4.0
2021-07-31,47,0.499,0.695,6.0,-9.452,1.0,0.1130,0.3330,0.000003,0.6590,0.2350,131.006,183920.0,4.0
2021-07-31,48,0.586,0.559,4.0,-6.725,1.0,0.0500,0.6390,0.000000,0.0872,0.4370,177.945,167480.0,4.0


The number of unique dates for the Viral 50

In [240]:
unique_dates = df_usa_50['date'].unique()
len(unique_dates)

1819

In [204]:
len(repeat_dates)

91225

In [205]:
miss_dates = df_usa_50['date'].value_counts().loc[lambda x: x<50].index.values

In [206]:
miss_val = 50 - df_usa_50['date'].value_counts().loc[lambda x: x<50].values

We add on extra to the negative class to compensate for some days have less than fifty for the Viral 50.

In [207]:
repeat_dates = np.repeat(unique_dates, 50)
print(len(repeat_dates))
for i in range(len(miss_dates)):
    for j in range(miss_val[i]):
        repeat_dates = np.append(repeat_dates,miss_dates[i])
print(len(repeat_dates))

90950
91225


## Add Negative Classes

In [208]:
track_df = pd.read_csv('tracks_features.csv')

In [209]:
track_df.drop(columns=['name', 'album', 'album_id', 'artists', 'artist_ids', 'track_number', 'disc_number', 'explicit', 'year', 'release_date'], inplace=True)

In [210]:
track_df

,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,7lmeHLHBe4nmXzuXc0HDjk,0.470,0.978,7,-5.399,1,0.0727,0.02610,0.000011,0.3560,0.503,117.906,210133,4.0
1,1wsRitfRRtWyEapl0q22o8,0.599,0.957,11,-5.764,1,0.1880,0.01290,0.000071,0.1550,0.489,103.680,206200,4.0
2,1hR0fIFK2qRG3f3RF70pb7,0.315,0.970,7,-5.424,1,0.4830,0.02340,0.000002,0.1220,0.370,149.749,298893,4.0
3,2lbASgTSoDO7MTuLAXlTW0,0.440,0.967,11,-5.830,0,0.2370,0.16300,0.000004,0.1210,0.574,96.752,213640,4.0
4,1MQTmpYOZ6fcMQc56Hdo7T,0.426,0.929,2,-6.729,1,0.0701,0.00162,0.105000,0.0789,0.539,127.059,205600,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1204020,0EsMifwUmMfJZxzoMPXJKZ,0.264,0.966,5,-6.970,0,0.0672,0.00935,0.002240,0.3370,0.415,159.586,276213,4.0
1204021,2WSc2TB1CSJgGE0PEzVeiu,0.796,0.701,11,-6.602,0,0.0883,0.10400,0.644000,0.0749,0.781,121.980,363179,4.0
1204022,6iProIgUe3ETpO6UT0v5Hg,0.785,0.796,9,-5.960,0,0.0564,0.03040,0.918000,0.0664,0.467,121.996,385335,4.0
1204023,37B4SXC8uoBsUyKCWnhPfX,0.665,0.856,6,-6.788,0,0.0409,0.00007,0.776000,0.1170,0.227,124.986,324455,4.0


In [211]:
track_df.to_csv('cleaned_features.csv', index=False)

In [212]:
track_ids = pd.DataFrame(track_df['id'])

Get the Ids of songs

In [213]:
df_usa_ids

,id
0,4Km5HrUvYTaSUfiSGPJeQR
1,343YBumqHu19cGoGARUTsd
2,5aAx2yezTd8zXrkmtKl66Z
3,7BKLCZ1jbUBVqRi2FVlTVw
4,6fujklziTHa8uoM5OQSfIo
...,...
14639,6jDOs0wGlRn4jCgJ4Z7HZv
14640,7eQHxigpuDJjCG50JyzU8v
14641,0kgRRHa5CMeCciIgYT67re
14642,7GLqR9ToJLb0PV3XyNAWNm


We remove any songs from the tracks list that occur in the USA list.

In [214]:
diff_newer = pd.concat([track_ids,df_usa_ids]).drop_duplicates(keep=False)

Merge so that only tracks found in the tracks data are included.

In [215]:
diff_newer = pd.merge(track_ids, df_usa_ids, on=['id','id'], how="outer", indicator=True
              ).query('_merge=="left_only"')

In [216]:
diff_newer.drop(columns='_merge', inplace=True)

In [217]:
diff_newer.to_csv('not_in_country.csv', index=False)

Sample as many songs are there are dates. This is one of severla augmentation methods. Another is to use only songs in the Charts Dataset.

In [218]:
df_selected = diff_newer.sample(n=len(repeat_dates))

In [219]:
df_selected['date'] = repeat_dates

In [220]:
df_selected

,id,date
469579,6iYrB76z6njk5qJiFKcQO6,2017-08-09
799993,5055VUxtmXo8m6JYzi5jJ4,2017-08-09
380124,5K4PTpycur6uSjU5OEicmk,2017-08-09
350550,5KjikkhdyCM7fyWz8JELdS,2017-08-09
216434,3Bx30CGhXnsTRazQRxiTuT,2017-08-09
...,...,...
749629,5ARX32nwfy4XfLpGV9485A,2019-10-30
189268,1oPNr88LrUAH0nErQWUYgv,2019-10-26
313202,2o1RcVgTyQJU05zmam7x09,2019-10-26
728404,55LedwlTAcicFnWzaKMhms,2019-10-20


In [221]:
df_usa_50['date']

800      2017-08-09
801      2017-08-01
802      2017-08-01
803      2017-08-01
804      2017-08-01
            ...    
454827   2021-07-31
454828   2021-07-31
454829   2021-07-31
454830   2021-07-31
454831   2021-07-31
Name: date, Length: 90675, dtype: datetime64[ns]

In [222]:
df_track_select = pd.DataFrame(track_df[track_df['id'].isin(df_selected['id'].values)])

In [223]:
df_select_merge = pd.merge(df_selected, df_track_select, how='inner', on = 'id')


In [224]:
df_select_merge

,id,date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,6iYrB76z6njk5qJiFKcQO6,2017-08-09,0.430,0.1430,9,-12.799,1,0.0417,0.863,0.000014,0.1060,0.3050,80.590,113583,4.0
1,5055VUxtmXo8m6JYzi5jJ4,2017-08-09,0.748,0.5960,5,-10.169,0,0.0768,0.742,0.906000,0.9100,0.7950,128.366,208212,4.0
2,5K4PTpycur6uSjU5OEicmk,2017-08-09,0.198,0.0955,8,-27.071,0,0.0394,0.901,0.949000,0.0841,0.0461,143.768,802987,4.0
3,5KjikkhdyCM7fyWz8JELdS,2017-08-09,0.451,0.2190,2,-24.456,1,0.5410,0.851,0.000000,0.4930,0.2440,138.605,52907,3.0
4,3Bx30CGhXnsTRazQRxiTuT,2017-08-09,0.543,0.4380,8,-14.032,1,0.0375,0.847,0.077300,0.1580,0.3970,131.253,317453,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91220,5ARX32nwfy4XfLpGV9485A,2019-10-30,0.628,0.3720,3,-10.905,1,0.0317,0.742,0.865000,0.1680,0.6830,99.271,294573,4.0
91221,1oPNr88LrUAH0nErQWUYgv,2019-10-26,0.803,0.1840,0,-22.811,1,0.0872,0.112,0.866000,0.0840,0.5350,144.257,196827,4.0
91222,2o1RcVgTyQJU05zmam7x09,2019-10-26,0.256,0.0600,5,-22.361,0,0.0514,0.995,0.660000,0.2090,0.3900,81.570,133413,4.0
91223,55LedwlTAcicFnWzaKMhms,2019-10-20,0.536,0.6780,11,-12.725,0,0.0609,0.210,0.879000,0.0734,0.7830,126.135,124680,4.0


In [225]:
df_select_merge['rank'] = 250

In [226]:
df_select_merge.drop(columns='id', inplace=True)

In [227]:
col = df_select_merge.pop("rank")
df_select_merge.insert(0, col.name, col)

In [228]:
df_select_merge

,rank,date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,250,2017-08-09,0.430,0.1430,9,-12.799,1,0.0417,0.863,0.000014,0.1060,0.3050,80.590,113583,4.0
1,250,2017-08-09,0.748,0.5960,5,-10.169,0,0.0768,0.742,0.906000,0.9100,0.7950,128.366,208212,4.0
2,250,2017-08-09,0.198,0.0955,8,-27.071,0,0.0394,0.901,0.949000,0.0841,0.0461,143.768,802987,4.0
3,250,2017-08-09,0.451,0.2190,2,-24.456,1,0.5410,0.851,0.000000,0.4930,0.2440,138.605,52907,3.0
4,250,2017-08-09,0.543,0.4380,8,-14.032,1,0.0375,0.847,0.077300,0.1580,0.3970,131.253,317453,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91220,250,2019-10-30,0.628,0.3720,3,-10.905,1,0.0317,0.742,0.865000,0.1680,0.6830,99.271,294573,4.0
91221,250,2019-10-26,0.803,0.1840,0,-22.811,1,0.0872,0.112,0.866000,0.0840,0.5350,144.257,196827,4.0
91222,250,2019-10-26,0.256,0.0600,5,-22.361,0,0.0514,0.995,0.660000,0.2090,0.3900,81.570,133413,4.0
91223,250,2019-10-20,0.536,0.6780,11,-12.725,0,0.0609,0.210,0.879000,0.0734,0.7830,126.135,124680,4.0


In [229]:
df_select_merge['class'] = 0

In [230]:
df_select_merge = df_select_merge.set_index('date')

Add in the class value for the target class.

In [231]:
df_usa_50_dex['class'] = 1

In [232]:
df_select_merge

,rank,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,class
date,,,,,,,,,,,,,,,
2017-08-09,250,0.430,0.1430,9,-12.799,1,0.0417,0.863,0.000014,0.1060,0.3050,80.590,113583,4.0,0
2017-08-09,250,0.748,0.5960,5,-10.169,0,0.0768,0.742,0.906000,0.9100,0.7950,128.366,208212,4.0,0
2017-08-09,250,0.198,0.0955,8,-27.071,0,0.0394,0.901,0.949000,0.0841,0.0461,143.768,802987,4.0,0
2017-08-09,250,0.451,0.2190,2,-24.456,1,0.5410,0.851,0.000000,0.4930,0.2440,138.605,52907,3.0,0
2017-08-09,250,0.543,0.4380,8,-14.032,1,0.0375,0.847,0.077300,0.1580,0.3970,131.253,317453,4.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-10-30,250,0.628,0.3720,3,-10.905,1,0.0317,0.742,0.865000,0.1680,0.6830,99.271,294573,4.0,0
2019-10-26,250,0.803,0.1840,0,-22.811,1,0.0872,0.112,0.866000,0.0840,0.5350,144.257,196827,4.0,0
2019-10-26,250,0.256,0.0600,5,-22.361,0,0.0514,0.995,0.660000,0.2090,0.3900,81.570,133413,4.0,0


In [233]:
df_usa_50_dex

,rank,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,class
date,,,,,,,,,,,,,,,
2017-08-09,32,0.595,0.610,1.0,-7.007,0.0,0.0330,0.0910,0.000000,0.1600,0.5040,99.529,218319.0,4.0,1
2017-08-01,1,0.810,0.342,1.0,-8.801,1.0,0.0892,0.7020,0.027400,0.1100,0.0881,77.515,199742.0,4.0,1
2017-08-01,2,0.653,0.910,10.0,-3.360,1.0,0.0566,0.0079,0.000000,0.3180,0.0921,126.047,234286.0,4.0,1
2017-08-01,3,0.922,0.583,5.0,-4.982,1.0,0.0753,0.0098,0.000000,0.0962,0.4940,136.043,261615.0,4.0,1
2017-08-01,4,0.867,0.545,2.0,-5.192,1.0,0.0625,0.0646,0.000289,0.0505,0.5250,139.943,162586.0,4.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-07-31,46,0.808,0.897,11.0,-3.712,0.0,0.0348,0.0469,0.000031,0.3640,0.5910,126.026,231041.0,4.0,1
2021-07-31,47,0.499,0.695,6.0,-9.452,1.0,0.1130,0.3330,0.000003,0.6590,0.2350,131.006,183920.0,4.0,1
2021-07-31,48,0.586,0.559,4.0,-6.725,1.0,0.0500,0.6390,0.000000,0.0872,0.4370,177.945,167480.0,4.0,1


Merge the two.

In [234]:
df_merged_class = pd.concat([df_usa_50_dex, df_select_merge])

In [235]:
df_merged_class

,rank,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,class
date,,,,,,,,,,,,,,,
2017-08-09,32,0.595,0.610,1.0,-7.007,0.0,0.0330,0.0910,0.000000,0.1600,0.5040,99.529,218319.0,4.0,1
2017-08-01,1,0.810,0.342,1.0,-8.801,1.0,0.0892,0.7020,0.027400,0.1100,0.0881,77.515,199742.0,4.0,1
2017-08-01,2,0.653,0.910,10.0,-3.360,1.0,0.0566,0.0079,0.000000,0.3180,0.0921,126.047,234286.0,4.0,1
2017-08-01,3,0.922,0.583,5.0,-4.982,1.0,0.0753,0.0098,0.000000,0.0962,0.4940,136.043,261615.0,4.0,1
2017-08-01,4,0.867,0.545,2.0,-5.192,1.0,0.0625,0.0646,0.000289,0.0505,0.5250,139.943,162586.0,4.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-10-30,250,0.628,0.372,3.0,-10.905,1.0,0.0317,0.7420,0.865000,0.1680,0.6830,99.271,294573.0,4.0,0
2019-10-26,250,0.803,0.184,0.0,-22.811,1.0,0.0872,0.1120,0.866000,0.0840,0.5350,144.257,196827.0,4.0,0
2019-10-26,250,0.256,0.060,5.0,-22.361,0.0,0.0514,0.9950,0.660000,0.2090,0.3900,81.570,133413.0,4.0,0


Sort by date.

In [236]:
df_merge_sorted = df_merged_class.sort_index()

In [237]:
df_merge_sorted

,rank,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,class
date,,,,,,,,,,,,,,,
2017-01-01,1,0.757,0.680,8.0,-8.988,0.0,0.0410,0.0168,0.015500,0.2750,0.249,122.772,281493.0,4.0,1
2017-01-01,250,0.399,0.781,11.0,-6.068,0.0,0.0944,0.4170,0.014600,0.3760,0.715,126.414,1052387.0,4.0,0
2017-01-01,250,0.493,0.192,7.0,-13.278,1.0,0.0311,0.8990,0.000000,0.6790,0.333,91.488,291133.0,4.0,0
2017-01-01,250,0.453,0.110,5.0,-25.176,0.0,0.0407,0.9720,0.848000,0.7420,0.234,107.737,441067.0,4.0,0
2017-01-01,250,0.734,0.826,8.0,-11.951,1.0,0.0609,0.0160,0.925000,0.0959,0.402,153.852,328907.0,4.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-31,250,0.487,0.273,7.0,-11.857,1.0,0.0345,0.9050,0.000000,0.0936,0.320,95.512,139733.0,4.0,0
2021-12-31,250,0.438,0.485,0.0,-12.575,1.0,0.0402,0.4420,0.004000,0.1200,0.171,109.986,262623.0,4.0,0
2021-12-31,250,0.156,0.305,5.0,-13.335,1.0,0.0359,0.5220,0.991000,0.1320,0.176,137.431,146506.0,4.0,0


In [238]:
df_merge_sorted.to_csv('usa_classes.csv')

In [242]:
df_merge_sorted.index.value_counts().loc[lambda x: x<50].index.values

array([], dtype='datetime64[ns]')

## Scaling and Output

In [280]:
from sklearn.preprocessing import StandardScaler

In [284]:
scaler = StandardScaler()
scaled = scaler.fit_transform(df_merge_sorted.drop(columns='class'))
np.save('usa_3d_scaled_flat.npy', scaled)

In [282]:
scaled_resized = np.resize(scaled,(1819, 100, 14))
print(scaled_resized.shape)
print(scaled_resized)
np.save('usa_3d_scale_minmax.npy',scaled_resized)

(1819, 100, 14)
[[[-1.21661822e+00  8.62784798e-01  4.71575466e-01 ...  1.17290415e-01
    4.67304261e-01  2.35286823e-01]
  [ 9.92895330e-01 -9.49267693e-01  8.76654963e-01 ...  2.38846282e-01
    6.75536242e+00  2.35286823e-01]
  [ 9.92895330e-01 -4.73477374e-01 -1.48564032e+00 ... -9.26848446e-01
    5.45936187e-01  2.35286823e-01]
  ...
  [ 9.92895330e-01  7.36244820e-01  1.24162758e+00 ... -7.73384999e-01
   -2.87472513e-01  2.35286823e-01]
  [ 9.92895330e-01 -3.01383003e-01 -8.55962289e-01 ... -2.76621190e-02
    5.26343460e-01  2.35286823e-01]
  [ 9.92895330e-01 -1.15173166e+00  6.56067118e-01 ... -1.39311299e+00
    1.01771839e-01  2.35286823e-01]]

 [[-9.23791121e-01 -8.32850913e-01  7.32270192e-01 ... -7.93377301e-01
   -2.50872777e-01  2.35286823e-01]
  [-9.32664669e-01 -8.88528503e-01 -1.14874252e+00 ...  2.90779540e-01
    1.91422144e+00  2.35286823e-01]
  [-9.41538217e-01 -5.24093365e-01 -1.77039917e+00 ... -4.41426006e-01
   -6.07546871e-01  2.35286823e-01]
  ...
  [ 9.9

In [252]:
sequences

In [253]:
sequences = df_merge_sorted.drop(columns='class').groupby(df_merge_sorted.index)
n3d = sequences.apply(lambda x : x.values.tolist()).tolist()

In [260]:
np.save('usa_3d.npy',np.array(n3d))

In [262]:
np.save('usa_3d_class.npy', df_merge_sorted['class'].to_numpy())